In [1]:
from pathlib import Path
import scanpy as sc
import re, sys
import pandas as pd

In [13]:
def parse_sam(file):
	result = set()
	with open(file, 'r') as inputFile:
		for line in inputFile.readlines():
			fields = line.split('\t')
			acc = fields[2]
			for item in fields[19:]:
				if item.find('CB:Z:') > -1:
					barcode = re.sub(r'CB:Z:', r'', item)
				if item.find('UB:Z:') > -1:
					umi = re.sub(r'UB:Z:', '', item)
			result.add(f'{acc}\t{barcode}\t{umi}')
	return result

def generate_table(data, target_set, target_name, suffix):
	filtered_data = []
	for item in data:
		if item.split('\t')[0] in target_set:
			filtered_data.append(item)

	with open('exp_aae_denv.txt', 'w') as outputFile:
		for item in filtered_data:
			outputFile.write(item)
	table = pd.read_table('exp_aae_denv.txt', header=None, delimiter='\t')
	table.columns = ['Accession', 'Barcode', 'UMI']
	table['Accession'] = table['Accession'].map(lambda x: target_name[x])
	table = table[table['Barcode'] != '-']
	table = table[table['UMI'] != '-']
	table = table.groupby(['Accession', 'Barcode'])['UMI'].count().reset_index()
	table['Barcode'] = table['Barcode'] + suffix
	Path('exp_aae_denv.txt').unlink()
	return table

def write2csv(adata, table, target_name, suffix):
	umap_df = pd.DataFrame(
	adata.obsm['X_umap'], 
	index=adata.obs_names,
	columns=['UMAP_1', 'UMAP_2']  # 列名可自定义
	)
	umap_df = pd.concat([adata.obs['cluster'], umap_df], axis=1).reset_index()
	umap_df.columns = ['Barcode', 'Cluster', 'UMAP_1', 'UMAP_2']
	umap_df.to_csv('temp.csv', header=True, index=False)
	umap_df = pd.read_csv('temp.csv', header=0)
	for name in target_name.values():
		temp_column = table[table['Accession'] == name][['Barcode', 'UMI']]
		umap_df = umap_df.merge(temp_column, how='left', on='Barcode').fillna(0)
	umap_df.columns = ['Barcode', 'Cluster', 'UMAP_1', 'UMAP_2'] + list(target_name.values())
	umap_df.to_csv(f'exp_{suffix}.csv', index=False)
	Path('temp.csv').unlink()
	print(umap_df)

In [4]:
data = parse_sam('exp_aae.sam')

In [5]:
target_set = ['MW174761.1', 'ON949933.1']
target_name = {
	'MW174761.1': 'Totichi',
	'ON949933.1': 'HKIFV'
}
table = generate_table(data, target_set, target_name, '-aae')

In [14]:
adata = sc.read('../../../processed_h5ad/aae_bl_denv.h5ad')
adata = adata[adata.obs['batch'] == 'aae']
write2csv(adata, table, target_name, 'aae')

                                Barcode      Cluster    UMAP_1     UMAP_2  \
0     CGATCGGTA_AACGTCCAA_AACAAGTGG-aae     ISC/EB-1 -0.519856  -3.288706   
1     CGCCAATGA_AACGTCCAA_AACAAGTGG-aae    EC-like-2  1.661915   2.763340   
2     TGCCTGATC_AACGTCCAA_AACAAGTGG-aae           EC -0.046305  17.824047   
3     GAGCAGCTT_ACACCAACG_AACAAGTGG-aae     ISC/EB-1 -1.831209  -1.088367   
4     TCCGTATCA_ACACCAACG_AACAAGTGG-aae  ISC/EB-prol  2.184209  -4.082183   
...                                 ...          ...       ...        ...   
9213  TACCTCTCC_TTACACGAC_TTGTGTACG-aae           EC  3.955785  12.939748   
9214  CGCAACTAC_TTCCAATCG_TTGTGTACG-aae    EC-like-1  4.432151  10.048210   
9215  TCGGTTCGT_TTCCAATCG_TTGTGTACG-aae         EE-1 -7.767694  -0.853176   
9216  CAGAGTACA_TTGAGACAG_TTGTGTACG-aae    EC-like-1  2.330974   5.904183   
9217  CGCAACTAC_TTGGTGACC_TTGTGTACG-aae  ISC/EB-prol  2.210938  -5.192821   

      Totichi  HKIFV  
0         0.0    3.0  
1         0.0    5.0  
2     